In [3]:
import torch
import json
import argparse
from tqdm import tqdm
from datasets import load_dataset
from PIL import Image
import sys
import numpy as np
import os
from w2s_utils import get_layer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import accelerate
from visualization import topk_intermediate_confidence_heatmap, accuracy_line
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
sys.path.append('/home/hyang/llava_paso/LLaVA')

from llava.mm_utils import get_model_name_from_path
from llava.constants import (
    IMAGE_TOKEN_INDEX,
    DEFAULT_IMAGE_TOKEN,
    DEFAULT_IM_START_TOKEN,
    DEFAULT_IM_END_TOKEN,
)
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model, load_pretrained_model_paper
from llava.utils import disable_torch_init
from llava.mm_utils import (
    tokenizer_image_token,
    get_model_name_from_path,
    KeywordsStoppingCriteria,
)

/home/hyang/anaconda3/envs/llava/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-12-19 18:38:36,196] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [7]:
def load_hades(mm_use_im_start_end=False, model_name="vicuna-7b-v1.5"):
    hades = load_dataset("Monosail/HADES")['test']
    hades = [item for item in hades if item['step'] == 5]
    malicious = []
    for item in hades:
        image = item['image'].convert('RGB')
        # image = Image.open("/home/hyang/llava_paso/blank_white_image.png")
        inst = item['instruction']

        # if item["category"] == "behavior":
        #     mask = "conduct this behavior on"
        # else:
        #     mask = f"this {item['category']}"
        # new_inst = f"The bottom of the image shows a {item['category']}. " + inst.replace(item["keywords"], mask)
        qs = inst
        qs = " "
        if mm_use_im_start_end:
            qs = (
                DEFAULT_IM_START_TOKEN
                + DEFAULT_IMAGE_TOKEN
                + DEFAULT_IM_END_TOKEN
                + "\n"
                + qs
            )
        else:
            qs = DEFAULT_IMAGE_TOKEN + "\n" + qs

        if "llama-2" in model_name.lower():
            conv_mode = "llava_llama_2"
        elif "v1" in model_name.lower():
            conv_mode = "llava_v1"
        elif "mpt" in model_name.lower():
            conv_mode = "mpt"
        else:
            conv_mode = "llava_v0"

        conv = conv_templates[conv_mode].copy()
        conv.append_message(conv.roles[0], qs)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()
        malicious.append({'id':item['id'], 'image':image,'prompt': prompt})
    return malicious

In [8]:
def load_normal(dataset_path, mm_use_im_start_end=False,model_name="vicuna-7b-v1.5"):
    with open(dataset_path, "r") as file:
        content = file.read()
    normal_dataset = json.loads(content)
    normal = []
    for item in tqdm(normal_dataset[:750]):
        image = Image.open("/home/hyang/llava_paso/coco/" + item['image']).convert('RGB').resize((1024, 1324))
        # image = Image.open("/home/hyang/llava_paso/blank_white_image.png")
        for conv_item in item['conversations']:
            if conv_item['from'] == 'human':
                # qs = conv_item['value']
                qs = " "
                if mm_use_im_start_end:
                    qs = (
                        DEFAULT_IM_START_TOKEN
                        + DEFAULT_IMAGE_TOKEN
                        + DEFAULT_IM_END_TOKEN
                        + "\n"
                        + qs
                    )
                elif DEFAULT_IMAGE_TOKEN not in qs:
                    qs = DEFAULT_IMAGE_TOKEN + "\n" + qs


                if "llama-2" in model_name.lower():
                    conv_mode = "llava_llama_2"
                elif "v1" in model_name.lower():
                    conv_mode = "llava_v1"
                elif "mpt" in model_name.lower():
                    conv_mode = "mpt"
                else:
                    conv_mode = "llava_v0"

                conv = conv_templates[conv_mode].copy()
                conv.append_message(conv.roles[0], qs)
                conv.append_message(conv.roles[1], None)
                prompt = conv.get_prompt()
                normal.append({'id':item['id'], 'image':image,'prompt': prompt})
    return normal


LLM Only

In [1]:
model_path = "lmsys/vicuna-7b-v1.5"

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_path, output_hidden_states=True, device_map="auto", cache_dir = "/home/hyang/llava_paso/.cache")
model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_path, cache_dir = "/home/hyang/llava_paso/.cache")

/home/hyang/anaconda3/envs/llava/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/home/hyang/anaconda3/envs/llava/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.69s/it]
/home/hyang/anaconda3/envs/llava/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is se

In [9]:
harmful_data = load_hades()

In [12]:
query = harmful_data[0]['prompt']

In [15]:
conv_mode = "llava_v1"
conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], query)
conv.append_message(conv.roles[1], None)
prompt = conv.get_prompt()
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
input_ids = input_ids.to(model.device)
with torch.no_grad():
    outputs = model(input_ids)